In [ ]:
import argparse
import copy
import os
import time

import cv2
import imageio
import numpy as np
import torch
import vispy
import yaml
from tqdm import tqdm

import sys
sys.path.append("/art_pipelines/libs/3d-photo-inpainting/")

import MiDaS.MiDaS_utils as MiDaS_utils
from MiDaS.monodepth_net import MonoDepthNet
from MiDaS.run import run_depth
from bilateral_filtering import sparse_bilateral_filtering
from boostmonodepth_utils import run_boostmonodepth, resize_depth,replace_ext, write_depth
from mesh import output_3d_photo, read_ply, write_ply
from networks import Inpaint_Color_Net, Inpaint_Depth_Net, Inpaint_Edge_Net
from utils import get_MiDaS_samples, read_MiDaS_depth

In [ ]:
workspace_dir = "/workspace"

depth_dir = f"{workspace_dir}/depth_folder"

os.makedirs(f"{workspace_dir}/mesh_folder", exist_ok=True)
os.makedirs(f"{workspace_dir}/video_folder", exist_ok=True)
os.makedirs(depth_dir, exist_ok=True)

In [ ]:
import pydantic
from typing import List, Dict, Any, Union

In [ ]:
class MidasConfig(pydantic.BaseModel):
    traj_types: List[str] = ['straight-line']
    x_shift_range: List[float] = [-0.04]
    y_shift_range: List[float] = [0.00]
    z_shift_range: List[float] = [0.00]
    video_postfix: List[Any] = ['left']
    depth_format: str = '.npy'
    num_frames: int = 4

In [ ]:
source_dir = "/workspace/source_images"

config = MidasConfig().dict()

sample_list = get_MiDaS_samples(
    image_folder=source_dir,
    depth_folder=depth_dir,
    config=config,
    specific='', 
    aft_certain=None
)

In [ ]:
BOOST_BASE = "/art_pipelines/libs/3d-photo-inpainting/BoostingMonocularDepth/"

os.system(
    f"cd {BOOST_BASE} && python run.py --Final --data_dir {source_dir}/  --output_dir {depth_dir} --depthNet 0"
)

In [ ]:
import numpy

In [ ]:
image_names = [sample["ref_img_fi"] for sample in sample_list]

for i, image_name in enumerate(image_names):
    img = imageio.imread(image_name)
    H, W = img.shape[:2]
    scale = 640.0 / max(H, W)

    # resize and save depth
    target_height, target_width = int(round(H * scale)), int(round(W * scale))
    depth = imageio.imread(os.path.join(depth_dir, image_name))
    depth = np.array(depth).astype(np.float32)
    depth = resize_depth(depth, target_width, target_height)
    output_path = os.path.join(depth_dir, replace_ext(os.path.basename(image_name), "npy"))
    print(f"Writing depth .npy to {output_path}")
    np.save(output_path, depth / 32768.0 - 1.0)
    output_path = os.path.join(depth_dir, os.path.basename(image_name))
    print(f"Writing depth to {output_path}")
    write_depth(output_path, depth)